In [1]:
#!pip install scikit-learn scipy matplotlib numpy
import sys
sys.path.insert(0, "..")

from base import *
from util import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import re
import numpy as np

from tensorflow.keras.utils import to_categorical

from fitter import Fitter, get_common_distributions, get_distributions
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.datasets import make_friedman3, make_s_curve
from sklearn.linear_model import BayesianRidge, ElasticNet
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.neural_network import MLPClassifier

#!pip install imbalanced-learn
#!pip install lightgbm
#!pip install wittgenstein --user
#!pip install explainerdashboard --user

from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


# for Box-Cox Transformation
from scipy import stats

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px

from tqdm import tqdm
import math

from sklearn.datasets import make_blobs
from matplotlib import pyplot
from pandas import DataFrame


In [2]:
#Collect data from sensors (i.e., time-series data) installed in the manufacturing system 
ai4i2020_encoded_balanced=pd.read_csv(r'G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\raw_data\\ai4i2020_encoded_balanced.csv')
ai4i2020_encoded_balanced.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,0,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,0,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,0,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,0,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [3]:
# Run in local
# Fold index
fold = 1

original_X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
original_X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")

In [4]:
ai4i2020_encoded_balanced = ai4i2020_encoded_balanced.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

## Hyper-parameter Tuning in FCMB Classifier

In [5]:
from sklearn.base import BaseEstimator, ClassifierMixin
from base.classifier import BaseFcmClassifier
from base.losses import MSE, LogLoss, SoftmaxCrossEntropy
from base.model import Model, AdditiveModel, Sigmoid, Tanh, Ident, Relu
from base.binary_classifier import FcmBinaryClassifier
import tensorflow as tf
import sklearn.preprocessing
import numpy as np


bs = 2**np.arange(4,13,4)
bs = np.append(bs, -1)

#we are training our model with the batch size ranging from 8 to 2048 with each batch size twice the size of the 
#previous batch size.
fcmb_param_grid = {'act': [None],
               'activation': ['sigmoid'],
               'activation_m': [1],
               'depth': [2, 3, 5],
               'epochs': [50, 100],
               'batch_size': bs, 
               'buffer_size': [1000],
               'training_loss': ['logloss'],
               'optimizer': ['rmsprop'],
               'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5],
               'model_type': ['basic'],
               'init_method': [None],
               'init_y_method': ['uniform'],
               'dropout_p': [0.0],
               'random_state': [None]}

grid = ParameterGrid(fcmb_param_grid)

display(str(len(grid))+" Combinations")

X = X_train.to_numpy()
y = y_train.to_numpy()

acc_list = []
auc_list = []
kappa_list = []

twf_list = []
hdf_list = []
pwf_list = []
osf_list = []
av_succ = []

comb_list = []

for comb in range(0,len(grid)):
    
    #Build the FCMB Classifier
    fcmb = FcmBinaryClassifier()
    display("Combination " + str(comb))
    fcmb.__init__(**grid[comb])
    fcmb.fit(X,y)
    
    #Predictions for the test set
    y_test_pred_fcmb = fcmb.predict(X_test)
    
    #Evaluation metrics
    accuracy_fcmb = accuracy_score(y_test, y_test_pred_fcmb)
    auc_fcmb = metrics.roc_auc_score(y_test, y_test_pred_fcmb)
    cohen_score_fcmb = cohen_kappa_score(y_test, y_test_pred_fcmb)

    
    #FCMB trajectory
    #For an output variable the constant 0.5 can be interpreted as undecided value, in between 0 and 1.
    init_y_test = np.ones((X_test.shape[0], fcmb.base_fcm.model.dim - X_test.shape[1])) * .5
    trj_fcmb_test = fcmb.base_fcm.model.trajectory(X_test, init_y_test)
    
    
    local_feature_importance_fcmb = pd.DataFrame(trj_fcmb_test[fcmb.depth], columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'])
    local_feature_importance_fcmb.drop('Machinefailure', inplace=True, axis=1)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_fcmb[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_list.append(accuracy_fcmb)
        auc_list.append(auc_fcmb)
        kappa_list.append(cohen_score_fcmb)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)
        
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;
        
        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_list.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_list.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_list.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_list.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_list.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_list.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_list.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_list.append(correct_explanations_OSF)
            osf_success = 0
    
        if denominator !=0:
            average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        else:
            average_success = 0
    
        av_succ.append(average_success)
        comb_list.append(comb)
    
    
    
result_df = pd.DataFrame({'Accuracy': acc_list, 'AUC': auc_list, 'Kappa':kappa_list, 'TWF':twf_list, 'HDF':hdf_list, 'PWF':pwf_list, 'OSF':osf_list, 'Average Success':av_succ, 'Combinations':comb_list})
final_df = result_df.sort_values(by=['AUC'], ascending=False)
final_df = final_df.reset_index(drop=True)
best_estimator_params = grid[final_df.loc[0,"Combinations"]]
display(best_estimator_params) 

'120 Combinations'

'Combination 0'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 50, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.001, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 570.4167244434357 sec



...

'Combination 119'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': -1, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.5, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 11.964931011199951 sec



{'training_loss': 'logloss',
 'random_state': None,
 'optimizer': 'rmsprop',
 'model_type': 'basic',
 'learning_rate': 0.01,
 'init_y_method': 'uniform',
 'init_method': None,
 'epochs': 100,
 'dropout_p': 0.0,
 'depth': 5,
 'buffer_size': 1000,
 'batch_size': 16,
 'activation_m': 1,
 'activation': 'sigmoid',
 'act': None}

In [6]:
final_df

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Combinations
0,0.962878,0.962402,0.925538,0.000000,0.000000,1.000000,1.000000,0.500000,26
1,0.961736,0.961310,0.923255,0.191176,0.259843,0.851852,0.996032,0.574726,37
2,0.960594,0.960536,0.921023,0.000000,1.000000,0.228346,1.000000,0.557087,27
3,0.960023,0.959408,0.919789,0.946667,0.830709,0.446970,0.884921,0.777316,12
4,0.957738,0.957277,0.915233,0.000000,0.045643,0.780303,0.964286,0.447558,17
...,...,...,...,...,...,...,...,...,...
107,0.523130,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,116
108,0.523130,0.500000,0.000000,1.000000,0.000000,1.000000,1.000000,0.750000,117
109,0.523130,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,85
110,0.522559,0.499719,-0.000588,0.000000,1.000000,0.410853,0.269841,0.420173,101


## Hyper-parameter Tuning in FCMMC Classifier

In [8]:
from sklearn.base import BaseEstimator, ClassifierMixin
from base.classifier import BaseFcmClassifier
from base.losses import MSE, LogLoss, SoftmaxCrossEntropy
from base.model import Model, AdditiveModel, Sigmoid, Tanh, Ident, Relu
from base.mc_classifier import FcmMulticlassClassifier
import tensorflow as tf
import sklearn.preprocessing
import numpy as np


bs = 2**np.arange(4,13,4)
bs = np.append(bs, -1)

#we are training our model with the batch size ranging from 8 to 2048 with each batch size twice the size of the 
#previous batch size.
fcmmc_param_grid = {'act': [None],
               'activation': ['sigmoid'],
               'activation_m': [1],
               'depth': [2, 3, 5],
               'epochs': [50, 100],
               'batch_size': bs, 
               'buffer_size': [1000],
               'training_loss': ['softmax'],
               'optimizer': ['rmsprop'],
               'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5],
               'model_type': ['basic'],
               'init_method': [None],
               'init_y_method': ['uniform'],
               'dropout_p': [0.0],
               'random_state': [None]}

grid_fcmmc = ParameterGrid(fcmmc_param_grid)

display(str(len(grid_fcmmc))+" Combinations")

X_fcmmc = X_train.to_numpy()
y_fcmmc = y_train.to_numpy()

acc_list_fcmmc = []
auc_list_fcmmc = []
kappa_list_fcmmc = []

twf_list_fcmmc = []
hdf_list_fcmmc = []
pwf_list_fcmmc = []
osf_list_fcmmc = []
av_succ_fcmmc = []

comb_list_fcmmc = []

for comb in range(0,len(grid_fcmmc)):
    
    #Build the FCMMC Classifier
    fcmmc = FcmMulticlassClassifier()
    display("Combination " + str(comb))
    fcmmc.__init__(**grid_fcmmc[comb])
    fcmmc.fit(X_fcmmc,y_fcmmc)
    
    #Predictions for the test set
    y_test_pred_fcmmc = fcmmc.predict(X_test)
    
    #Evaluation metrics
    accuracy_fcmmc = accuracy_score(y_test, y_test_pred_fcmmc)
    auc_fcmmc = metrics.roc_auc_score(y_test, y_test_pred_fcmmc)
    cohen_score_fcmmc = cohen_kappa_score(y_test, y_test_pred_fcmmc)

    
    #FCMMC trajectory
    #For an output variable the constant 0.5 can be interpreted as undecided value, in between 0 and 1.
    init_y_test = np.ones((X_test.shape[0], fcmmc.base_fcm.model.dim - X_test.shape[1])) * .5
    trj_fcmmc_test = fcmmc.base_fcm.model.trajectory(X_test, init_y_test)
    
    
    local_feature_importance_fcmmc = pd.DataFrame(trj_fcmmc_test[fcmmc.depth],  columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', '0', '1'])
    local_feature_importance_fcmmc.drop('0', inplace=True, axis=1)
    local_feature_importance_fcmmc.drop('1', inplace=True, axis=1)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_fcmmc[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_list_fcmmc.append(accuracy_fcmmc)
        auc_list_fcmmc.append(auc_fcmmc)
        kappa_list_fcmmc.append(cohen_score_fcmmc)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)
        
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;
        
        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_list_fcmmc.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_list_fcmmc.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_list_fcmmc.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_list_fcmmc.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_list_fcmmc.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_list_fcmmc.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_list_fcmmc.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_list_fcmmc.append(correct_explanations_OSF)
            osf_success = 0
        
        if denominator !=0:
            average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        else:
            average_success = 0
    
        av_succ_fcmmc.append(average_success)
        comb_list_fcmmc.append(comb)
    
    
    
result_df_fcmmc = pd.DataFrame({'Accuracy': acc_list_fcmmc, 'AUC': auc_list_fcmmc, 'Kappa':kappa_list_fcmmc, 'TWF':twf_list_fcmmc, 'HDF':hdf_list_fcmmc, 'PWF':pwf_list_fcmmc, 'OSF':osf_list_fcmmc, 'Average Success':av_succ_fcmmc, 'Combinations':comb_list_fcmmc})
final_df_fcmmc = result_df_fcmmc.sort_values(by=['AUC'], ascending=False)
final_df_fcmmc = final_df_fcmmc.reset_index(drop=True)
best_estimator_params_fcmmc = grid_fcmmc[final_df_fcmmc.loc[0,"Combinations"]]
display(best_estimator_params_fcmmc)

'120 Combinations'

'Combination 0'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 50, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.001, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 650.4616785049438 sec



...

'Combination 119'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': -1, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.5, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 13.151398658752441 sec



{'training_loss': 'softmax',
 'random_state': None,
 'optimizer': 'rmsprop',
 'model_type': 'basic',
 'learning_rate': 0.05,
 'init_y_method': 'uniform',
 'init_method': None,
 'epochs': 100,
 'dropout_p': 0.0,
 'depth': 2,
 'buffer_size': 1000,
 'batch_size': 16,
 'activation_m': 1,
 'activation': 'sigmoid',
 'act': None}

In [9]:
final_df_fcmmc

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Combinations
0,0.967447,0.966715,0.934668,0.0,0.000000,1.000000,0.757937,0.439484,7
1,0.962307,0.961114,0.924284,0.0,0.000000,0.985185,0.714286,0.424868,3
2,0.957738,0.957754,0.915314,0.0,0.780392,0.726562,0.674603,0.545389,34
3,0.957167,0.956519,0.914055,0.0,1.000000,0.179104,0.764000,0.485776,17
4,0.954883,0.953489,0.909337,0.0,0.822134,0.651852,0.773810,0.561949,18
...,...,...,...,...,...,...,...,...,...
107,0.523130,0.500000,0.000000,1.0,1.000000,0.000000,1.000000,0.750000,85
108,0.523130,0.500000,0.000000,0.0,1.000000,0.000000,0.000000,0.250000,70
109,0.523130,0.500000,0.000000,0.0,0.000000,1.000000,1.000000,0.500000,60
110,0.523130,0.500000,0.000000,0.0,1.000000,1.000000,0.000000,0.500000,119


## Calculate the evaluation metrics for all folds of the best FCMB estimator

In [10]:
acc_folds_best_estimator = []
auc_folds_best_estimator = []
kappa_folds_best_estimator = []
twf_folds_best_estimator = []
hdf_folds_best_estimator = []
pwf_folds_best_estimator = []
osf_folds_best_estimator = []
av_folds_best_estimator = []
fold_list = []


for fold in range(1,11):
    
    #Dataset for each fold
    original_X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
    original_X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

    X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
    X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")
    
    original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    #Dataframes to numpy
    X_train_numpy = X_train.to_numpy()
    y_train_numpy = y_train.to_numpy()
    X_test_numpy = X_test.to_numpy()
    y_test_numpy = y_test.to_numpy()
    
    
    #Build the FCMB Classifier
    fcmb = FcmBinaryClassifier()
    fcmb.__init__(**grid[final_df.loc[0,"Combinations"]])
    fcmb.fit(X_train_numpy,y_train_numpy)
    
    #Predictions for the test set
    y_test_pred_fcmb = fcmb.predict(X_test)
    
    #Evaluation metrics
    accuracy_fcmb = accuracy_score(y_test, y_test_pred_fcmb)
    auc_fcmb = metrics.roc_auc_score(y_test, y_test_pred_fcmb)
    cohen_score_fcmb = cohen_kappa_score(y_test, y_test_pred_fcmb)
    
    #FCMB trajectory
    #For an output variable the constant 0.5 can be interpreted as undecided value, in between 0 and 1.
    init_y_test = np.ones((X_test.shape[0], fcmb.base_fcm.model.dim - X_test.shape[1])) * .5
    trj_fcmb_test = fcmb.base_fcm.model.trajectory(X_test, init_y_test)
    
    
    local_feature_importance_fcmb = pd.DataFrame(trj_fcmb_test[fcmb.depth], columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'])
    local_feature_importance_fcmb.drop('Machinefailure', inplace=True, axis=1)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_fcmb[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_folds_best_estimator.append(accuracy_fcmb)
        kappa_folds_best_estimator.append(cohen_score_fcmb)
        auc_folds_best_estimator.append(auc_fcmb)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)

    
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;

        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_folds_best_estimator.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_folds_best_estimator.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_folds_best_estimator.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_folds_best_estimator.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_folds_best_estimator.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_folds_best_estimator.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_folds_best_estimator.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_folds_best_estimator.append(correct_explanations_OSF)
            osf_success = 0
        
        if denominator !=0:
            average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        else:
            average_success = 0
        
        av_folds_best_estimator.append(average_success)
        fold_list.append(fold)

result_df_best_estimator = pd.DataFrame({'Accuracy': acc_folds_best_estimator, 'AUC': auc_folds_best_estimator, 'Kappa':kappa_folds_best_estimator, 'TWF':twf_folds_best_estimator, 'HDF':hdf_folds_best_estimator, 'PWF':pwf_folds_best_estimator, 'OSF':osf_folds_best_estimator, 'Average Success':av_folds_best_estimator, 'Folds':fold_list})


-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1799.400580406189 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1800.6756253242493 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


...

Learning time 1800.6490757465363 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1795.2659068107605 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 5, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1794.3789658546448 sec



In [11]:
result_df_best_estimator

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Folds
0,0.927470,0.925223,0.854018,0.000000,1.000000,0.014815,0.000000,0.253704,1
1,0.952027,0.951395,0.903747,0.953846,0.000000,0.284884,1.000000,0.559682,2
2,0.954883,0.953754,0.909385,0.000000,1.000000,0.479532,0.008889,0.372105,3
3,0.940034,0.937973,0.879345,0.000000,1.000000,0.000000,0.227273,0.306818,4
4,0.958310,0.957293,0.916285,0.632653,1.000000,0.000000,1.000000,0.658163,5
5,0.950314,0.951399,0.900652,0.000000,1.000000,0.033149,0.918919,0.488017,6
6,0.921759,0.921565,0.843174,0.000000,1.000000,0.000000,0.000000,0.250000,7
7,0.952000,0.950643,0.903557,0.000000,0.008230,0.300000,0.339367,0.161899,8
8,0.930286,0.929542,0.860099,0.000000,1.000000,0.478571,0.012195,0.372692,9
9,0.944000,0.942696,0.887488,0.364865,0.982533,0.000000,0.456067,0.450866,10


## Calculate the evaluation metrics for all folds of the best FCMMC estimator

In [13]:
acc_folds_best_estimator_fcmmc = []
auc_folds_best_estimator_fcmmc = []
kappa_folds_best_estimator_fcmmc = []
twf_folds_best_estimator_fcmmc = []
hdf_folds_best_estimator_fcmmc = []
pwf_folds_best_estimator_fcmmc = []
osf_folds_best_estimator_fcmmc = []
av_folds_best_estimator_fcmmc = []
fold_list_fcmmc = []


for fold in range(1,11):
    
    #Dataset for each fold
    original_X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
    original_X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

    X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
    X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")
    
    original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    #Dataframes to numpy
    X_train_numpy = X_train.to_numpy()
    y_train_numpy = y_train.to_numpy()
    X_test_numpy = X_test.to_numpy()
    y_test_numpy = y_test.to_numpy()
    
    
    #Build the FCMMC Classifier
    
    fcmmc = FcmMulticlassClassifier()
    fcmmc.__init__(**grid_fcmmc[final_df_fcmmc.loc[0,"Combinations"]])
    fcmmc.fit(X_train_numpy,y_train_numpy)
        
    #Predictions for the test set
    y_test_pred_fcmmc = fcmmc.predict(X_test)
    
    #Evaluation metrics
    accuracy_fcmmc = accuracy_score(y_test, y_test_pred_fcmmc)
    auc_fcmmc = metrics.roc_auc_score(y_test, y_test_pred_fcmmc)
    cohen_score_fcmmc = cohen_kappa_score(y_test, y_test_pred_fcmmc)
    
    #FCMMC trajectory
    #For an output variable the constant 0.5 can be interpreted as undecided value, in between 0 and 1.
    init_y_test = np.ones((X_test.shape[0], fcmmc.base_fcm.model.dim - X_test.shape[1])) * .5
    trj_fcmmc_test = fcmmc.base_fcm.model.trajectory(X_test, init_y_test)
    
    local_feature_importance_fcmmc = pd.DataFrame(trj_fcmmc_test[fcmmc.depth],  columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', '0', '1'])
    local_feature_importance_fcmmc.drop('0', inplace=True, axis=1)
    local_feature_importance_fcmmc.drop('1', inplace=True, axis=1)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_fcmmc[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_folds_best_estimator_fcmmc.append(accuracy_fcmmc)
        kappa_folds_best_estimator_fcmmc.append(cohen_score_fcmmc)
        auc_folds_best_estimator_fcmmc.append(auc_fcmmc)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)

    
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmmc.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmmc.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;

        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_folds_best_estimator_fcmmc.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_folds_best_estimator_fcmmc.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_folds_best_estimator_fcmmc.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_folds_best_estimator_fcmmc.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_folds_best_estimator_fcmmc.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_folds_best_estimator_fcmmc.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_folds_best_estimator_fcmmc.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_folds_best_estimator_fcmmc.append(correct_explanations_OSF)
            osf_success = 0
    
        if denominator !=0:
            average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        else:
            average_success = 0
        
        av_folds_best_estimator_fcmmc.append(average_success)
        fold_list_fcmmc.append(fold)

result_df_best_estimator_fcmmc = pd.DataFrame({'Accuracy': acc_folds_best_estimator_fcmmc, 'AUC': auc_folds_best_estimator_fcmmc, 'Kappa':kappa_folds_best_estimator_fcmmc, 'TWF':twf_folds_best_estimator_fcmmc, 'HDF':hdf_folds_best_estimator_fcmmc, 'PWF':pwf_folds_best_estimator_fcmmc, 'OSF':osf_folds_best_estimator_fcmmc, 'Average Success':av_folds_best_estimator_fcmmc, 'Folds':fold_list_fcmmc})


-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.05, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1417.5360944271088 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.05, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1666.93971657753 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.05, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


...

Learning time 1233.482632637024 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.05, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1358.840985774994 sec

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 16, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 100, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.05, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'softmax'}
-----------------------------------------


C:\Users\Marios\Jupyter_Notebooks_my_papers\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1417.0169944763184 sec



In [14]:
result_df_best_estimator_fcmmc

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Folds
0,0.929754,0.930954,0.859587,0.492754,0.000000,0.054795,0.908730,0.364070,1
1,0.957738,0.956271,0.915062,0.986842,0.000000,1.000000,1.000000,0.746711,2
2,0.964592,0.964039,0.928964,0.000000,0.958678,0.289017,1.000000,0.561924,3
3,0.922901,0.923557,0.845725,0.000000,0.109649,0.932432,0.933884,0.493991,4
4,0.930897,0.931834,0.861797,0.000000,0.995885,0.540541,0.926087,0.615628,5
5,0.914906,0.915915,0.829852,0.000000,0.241228,0.989899,0.909910,0.535259,6
6,0.924615,0.925353,0.849177,0.959459,1.000000,0.701031,1.000000,0.915123,7
7,0.917714,0.919113,0.835600,0.000000,1.000000,0.957447,0.638009,0.648864,8
8,0.931429,0.932619,0.862923,0.000000,0.000000,0.956989,0.991870,0.487215,9
9,0.926857,0.928575,0.853941,0.000000,0.940171,0.453488,0.379167,0.443206,10
